### Circuit reduction
This notebook is intended for applying a neuron reduction algorithm by Oren Amsalem to a neuronal circuit. For details on the algorithm refer to https://doi.org/10.1038/s41467-019-13932-6.

**install helper API to work with jobs on PizDaint**

In [ ]:
!pip install -q --index-url https://bbpteam.epfl.ch/repository/devpi/bbprelman/dev/+simple/ hbp-jupyter-helpers

**Import necessary job API**

In [ ]:
# Disable warning about https unicore (temporary solution)
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from hbp_jupyter_helpers.pizdaint.job_manager import JobManager
from hbp_jupyter_helpers.pizdaint.sonata_network_reduction import run_reduction_job

**Create a job manager, so we can create a job to run reduction**

In [ ]:
job_manager = JobManager(
    'https://bspsa.cineca.it/advanced/pizdaint/rest/core',
    oauth.get_token())

**The following code will apply the neuron reduction algorithm to every neuron of the circuit. You may adjust parameters for your needs.**

In [ ]:
# directory with original BlueConfig circuit
original_circuit_dir = '/store/hbp/ich002/public/CA1.O0/20191017/'
# specify the SONATA config file of a circuit that you want to reduce
original_sonata_config = original_circuit_dir + 'sonata/circuit_config.json'
# define a folder where to save the reduced SONATA circuit
reduced_sonata_dir = 'sonata-reduced/'
# specify reduction parameters for the neuron reduction algorithm
reduction_kwargs = {'reduction_frequency': 0}
# run a reduction job
job_url, job_dir_url = run_reduction_job(
    job_manager, original_sonata_config, reduced_sonata_dir, reduction_kwargs)

**Wait for results.**

While the reduction is in process, you will see the log output of the neuron reduction algorithm. It may be quite lengthy. Upon success, you must see a line with "Reduction finished successfully". Upon failure, you must see a line with "Reduction failed" and the error output of the neuron reduction algorithm.

In [ ]:
status = job_manager.wait_for_job(job_url)
if status == 'SUCCESSFUL':
    print('\nReduction finished successfully. See "{}" for results.'.format(reduced_sonata_dir))
if status == 'FAILED':
    print('\nReduction failed')
    print('--------------------------------------')
    print(job_manager.get_file('stderr', job_dir_url))

**Simulate the reduced circuit.**

In order to simulate, execute the next cell and click upon the link "simulate the reduced circuit" below it.

In [ ]:
import base64
import json
import re

phys_dir = job_manager.get_job_phys_dir(job_url)
if phys_dir is None:
    raise ValueError('Verify that job has been created!')

simulation_config = {
  "baseConfig":"hippo_mooc_sa_microcircuit",
  "computer": "service_account",
  "circuitConfig":{
    "paths":{
      "MorphologyPath": phys_dir + reduced_sonata_dir + "components/morphologies",
      "MorphologyType": "swc",
      "METypePath": phys_dir + reduced_sonata_dir + "components/biophysical_neuron_models/hoc",
      "MEComboInfoFile": None,
      "CircuitPath": original_circuit_dir,
      "nrnPath": phys_dir + reduced_sonata_dir + "networks/edges/functional/default/edges.h5",
      "TargetFile": original_circuit_dir + "user.target",
      "CellLibraryFile": phys_dir + reduced_sonata_dir + "networks/nodes/All/nodes.h5",
    },
  },
  "projectionConfig": None,
  "simulationConfig":{
    "script":[
      ". /etc/profile",
      "export MODULEPATH=/apps/hbp/ich002/hbp-spack-deployments/softwares/25-03-2020/install/modules/tcl/cray-cnl7-haswell:$MODULEPATH",
      "module load daint-mc cray-python/3.6.5.7 PyExtensions/3.6.5.7-CrayGNU-19.10",
      "module use /apps/hbp/ich002/hbp-spack-deployments/softwares/25-03-2020/install/modules/tcl/cray-cnl7-haswell",
      "module load neuron/7.8.0b",
      "module load neurodamus-hippocampus/0.4",
      "module load py-bluepy/0.14.6",
      "export NFRAME=1000",
      "export HOC_LIBRARY_PATH=/store/hbp/ich002/antonel/O0/20191017/sonata_reduced/hoc",
      "export HDF5_USE_FILE_LOCKING=FALSE",
      "python /apps/hbp/ich002/home/antonel/create_replay_piz_daint_0.19.0.py -vv",
      "python /apps/hbp/ich002/home/antonel/simulation_launch_piz_daint_0.19.0.py -vv",
      "echo 'testing...'",
    ],
  },
}

simulation_url = 'https://bbp.epfl.ch/public/simulationapp/index.html#/circuits/hbp_dynamic_circuit?'
simulation_url += base64.urlsafe_b64encode(json.dumps(simulation_config).encode()).rstrip(b"=").decode()
get_ipython().run_cell_magic(
    u'HTML', u'', u'<a href="{}">simulate the reduced circuit</a>'.format(simulation_url))